## Imports

In [1]:
import pickle
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

#mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    categories_columns = ['PULocationID', 'DOLocationID']
    df[categories_columns] = df[categories_columns].astype('str')
    
    df['duration'] = df['lpep_dropoff_datetime'] -  df['lpep_pickup_datetime']
    
    df['duration'] = (df['duration'].dt.seconds /60).astype('float64').round(4) # duration in minutes
    
    # just o get the same result --> but is probably wrong do this in validation data
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [6]:
def select_data(data):
    
    data['PU_DO'] = data['PULocationID'] + '_' +data['DOLocationID']
    
    categories_columns = ['PU_DO']
    #categories_columns = ['PULocationID', 'DOLocationID']
    numerical_columns = ['trip_distance']
    
    dict_data = data[categories_columns + numerical_columns].to_dict(orient='records')
    
    return dict_data

## Train Data

In [7]:
train_path = '../../datasets/green_tripdata_2021-01.parquet'
df_train = read_data(train_path)

#df_train = df_train.loc[df_train['duration'].between(1,60)]

y_train = df_train['duration']

dict_train = select_data(df_train)

## Validation Data

In [8]:
val_path = '../../datasets/green_tripdata_2021-02.parquet'
df_validation = read_data(val_path)

y_validation = df_validation['duration']

dict_validation = select_data(df_validation)

### Random Forest

In [9]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_validation)

    rmse = mean_squared_error(y_pred, y_validation, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.755822890408456


/home/miguelzph/anaconda3/envs/taxi_project/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Prediction 

#### MLflowl + pyfunc  Model

In [ ]:
#import mlflow
logged_model = 'runs:/ec83b5fdf5ef4ff5be337123791ffe4f/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(x_validation)

In [ ]:
mean_squared_error(y_validation, y_pred, squared=False)

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'f6705545dd7a4d97910dcf24be62b0de'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# dict vectorizer
path_dv = client.download_artifacts(run_id=RUN_ID,
                          path='preprocessor/dict_vectorizer.bin')

with open(path_dv, 'rb') as f_in:
    dv = pickle.load(f_in)

# model 
logged_model = f'runs:/{RUN_ID}/models_mlflow'

model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
dv